In [1]:
import pandas_datareader.data as web
import datetime
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
start = datetime.datetime(2021,1,1)

end = datetime.datetime(2021,1,21)

In [3]:
ceva = web.DataReader('CEVA', 'yahoo', start , end)
google = web.DataReader('GOOGL', 'yahoo', start, end)
tesla = web.DataReader('TSLA', 'yahoo', start, end)
zom = web.DataReader('ZOM', 'yahoo', start, end)


In [4]:
stonks = pd.concat([ceva['Close'], google['Close'], tesla['Close'], zom['Close']], axis=1)
stonks.columns = ['CEVA', 'GOOGLE', 'TESLA', 'ZOMEDICA']

stonks

,CEVA,GOOGLE,TESLA,ZOMEDICA
Date,,,,
2021-01-04,44.720001,1726.130005,729.770020,0.350
2021-01-05,46.430000,1740.050049,735.109985,0.439
2021-01-06,48.160000,1722.880005,755.979980,0.410
2021-01-07,50.700001,1774.339966,816.039978,0.400
2021-01-08,50.910000,1797.829956,880.020020,0.484
2021-01-11,54.139999,1756.290039,811.190002,0.930
2021-01-12,55.540001,1737.430054,849.440002,1.300
2021-01-13,57.119999,1747.250000,854.409973,0.976
2021-01-14,55.669998,1730.920044,845.000000,1.010


In [5]:
returns = stonks/stonks.shift(1)
print(returns)

# the correct function for returns is below !!!!!!!!!!!!!!
# returns = stonks.pct_change()
# print(returns)


                CEVA    GOOGLE     TESLA  ZOMEDICA
Date                                              
2021-01-04       NaN       NaN       NaN       NaN
2021-01-05  1.038238  1.008064  1.007317  1.254286
2021-01-06  1.037260  0.990132  1.028390  0.933941
2021-01-07  1.052741  1.029869  1.079447  0.975610
2021-01-08  1.004142  1.013239  1.078403  1.210000
2021-01-11  1.063445  0.976894  0.921786  1.921488
2021-01-12  1.025859  0.989261  1.047153  1.397849
2021-01-13  1.028448  1.005652  1.005851  0.750769
2021-01-14  0.974615  0.990654  0.988987  1.034836
2021-01-15  0.968744  0.998093  0.977704  0.965347
2021-01-19  1.173002  1.032907  1.022260  1.107692
2021-01-20  1.041891  1.053573  1.006986  0.944444
2021-01-21  0.977242  1.002170  0.993580  0.980392


In [6]:
logReturns = np.log(returns)
logReturns

,CEVA,GOOGLE,TESLA,ZOMEDICA
Date,,,,
2021-01-04,NaN,NaN,NaN,NaN
2021-01-05,0.037525,0.008032,0.007291,0.226566
2021-01-06,0.036583,-0.009917,0.027995,-0.068342
2021-01-07,0.051397,0.029431,0.076448,-0.024693
2021-01-08,0.004133,0.013152,0.075481,0.190620
2021-01-11,0.061514,-0.023377,-0.081442,0.653100
2021-01-12,0.025530,-0.010797,0.046075,0.334935
2021-01-13,0.028051,0.005636,0.005834,-0.286657
2021-01-14,-0.025713,-0.009390,-0.011075,0.034243


In [7]:
pBar = logReturns.mean()
Sigma = logReturns.cov()

In [8]:
rMin = 0.02

def riskFunction(w):
    return np.dot(w.T,np.dot(Sigma, w))

w0 = [0.25, 0.25, 0.25, 0.25]
bounds = ((0,1),(0,1),(0,1),(0,1))

def checkMinimumReturn(w):
    RHS = rMin - np.sum(pBar * w)
    return RHS
def checkSumToOne(w):
    return np.sum(w) - 1

constraints = ({'type':'eq', 'fun':checkMinimumReturn},{'type':'eq', 'fun':checkSumToOne})

w_opt = minimize(riskFunction, w0, method='SLSQP', bounds=bounds, constraints=constraints)

In [9]:
w_scipy = w_opt.x

w_scipy

array([0.26696774, 0.34362012, 0.32806326, 0.06134888])

In [10]:
risk_scipy = riskFunction(w_scipy)

risk_scipy

0.0007318892616290768

In [11]:
np.sum(pBar * w_scipy)

0.019999999955618675

In [12]:
np.sum(w_scipy)

1.0

In [19]:
def ourMarkowitzSingleEquationSolver(rMin, Sigma, pBar):
    N = len(Sigma)
    o = np.ones(N)
    SigmaInv = np.linalg.inv(Sigma)
    a = np.dot(pBar.T, np.dot(SigmaInv, pBar))
    b = np.dot(pBar.T, np.dot(SigmaInv, o))
    c = np.dot(o.T, np.dot(SigmaInv, o))
    return (1/ (a*c - b**2)) * np.dot(SigmaInv, ((c * rMin - b)* pBar +(a - b * rMin)*o))

In [21]:
w_ourSingleEquation = ourMarkowitzSingleEquationSolver(rMin, Sigma, pBar)
print(w_ourSingleEquation)
risk_ourSingleEquation = riskFunction(w_ourSingleEquation)
print(risk_ourSingleEquation)

[0.24473372 0.44492543 0.23700196 0.07333889]
0.0007099864708157671


In [24]:
np.sum(pBar * w_ourSingleEquation)

0.019999999999999997

In [25]:
np.sum(w_ourSingleEquation)

0.9999999999999997